# Transfer Learning

In [1]:
import sys

import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

from Helpers import get_confusion_matrix_for_model_and_data

sys.path.append("../src")

import keras
import tensorflow as tf
import numpy as np

from TrainProdecure import train_single_model
from ModelBuilder import get_FCN
from LoadData import get_all_datasets_test_train_np_arrays

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from LoadData import get_fcn_datasets_test_train_np_arrays, DATASETS_FCN

EPOCHS_PER_TRAINING = 20
# USED_DATASETS = get_fcn_datasets_test_train_np_arrays('../datasets/')
USED_DATASETS = get_all_datasets_test_train_np_arrays('../datasets/', ds_names=DATASETS_FCN[:3])
GENERATE_MODEL = get_FCN
MODEL_NAME = 'FCN'
FINE_TUNE_LEARNING_RATE = 10e-5

In [3]:
def train_model_from_scratch(x_train, y_train, x_test, y_test, dataset_name):
    """
    Function to train a model from scratch on a given dataset
    :return: the trained model, the loss, the accuracy, and the history
    """
    input_size = x_train.shape[1]
    output_size = len(np.unique(y_train))

    scratch_model = GENERATE_MODEL(input_size, output_size)
    # scratch_model.summary()

    return train_single_model(
        scratch_model, x_train, y_train, x_test, y_test,
        epochs=EPOCHS_PER_TRAINING, batch_size=None, model_name=MODEL_NAME, dataset_name=dataset_name
    )

In [4]:
def transfer_learning_model_one_dataset(x_train, y_train, x_test, y_test, dataset_name):
    input_size = x_train.shape[1]
    output_size = len(np.unique(y_train))

    model = GENERATE_MODEL(input_size, output_size)

    return train_single_model(
        model, x_train, y_train, x_test, y_test,
        epochs=EPOCHS_PER_TRAINING, batch_size=25, model_name='FCN',
        dataset_name=dataset_name
    )

In [5]:
def two_confusion_matrices(confusion_matrix_1, model_name_1, confusion_matrix_2, model_name_2, dataset_name):
    plt.figure(figsize=(20, 10))
    plt.title(f'Transfer Learning for {dataset_name}')
    plt.subplot(1, 2, 1)
    sns.heatmap(confusion_matrix_1, annot=True, fmt="d")
    plt.title(f"Confusion Matrix for {model_name_1}")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.subplot(1, 2, 2)
    sns.heatmap(confusion_matrix_2, annot=True, fmt="d")
    plt.title(f"Confusion Matrix for {model_name_2}")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [6]:
def transfer_learning_for_all_datasets(used_datasets):
    """
    Function to train a model from scratch on a given dataset
    :return: the trained model, the loss, the accuracy, and the history
    """
    # Target ds for which it is fine-tuned:
    for dataset_name, data_dict in tqdm(used_datasets.items(), unit='datasets', desc='Transfer Learning for all datasets'):
        print(f"Training model targeting dataset {dataset_name}")

        x_train, y_train = data_dict['train_data']
        x_test, y_test = data_dict['test_data']

        # ds for pre-training:
        datasets_subset = {name: value for name, value in used_datasets.items() if name != dataset_name}
        for pre_train_ds_name, pre_train_data_dict in tqdm(datasets_subset.items(), unit='datasets', desc='Pre-training on different datasets'):
            print(f"Pre-training model on dataset {dataset_name}")

            pre_train_x_train, pre_train_y_train = pre_train_data_dict['train_data']
            pre_train_x_test, pre_train_y_test = pre_train_data_dict['test_data']

            # Training a model from scratch as a baseline:
            scratch_model, scratch_loss, scratch_accuracy, scratch_history = train_model_from_scratch(
                x_train, y_train, x_test, y_test, dataset_name
            )

            # Pre-training a model:
            pre_trained_model, _, _, _ = transfer_learning_model_one_dataset(
                pre_train_x_train, pre_train_y_train, pre_train_x_test, pre_train_y_test, pre_train_ds_name
            )

            # Fine-tuning:
            input_size = x_train.shape[1]
            output_size = len(np.unique(y_train))
            headless_model_layers = pre_trained_model.layers[:-1]
            fine_tuned_model = keras.Sequential([*headless_model_layers, keras.layers.Dense(output_size, activation='softmax')])
            fine_tuned_model.build(input_shape=(None, input_size, 1))
            train_single_model(
                fine_tuned_model, x_train, y_train, x_test, y_test,
                epochs=EPOCHS_PER_TRAINING, batch_size=25, model_name='FCN', dataset_name=dataset_name
            )

            # Compare confusion matrices:
            from_scratch_confusion_matrix = get_confusion_matrix_for_model_and_data(scratch_model, x_test, y_test)
            transfer_learned_confusion_matrix = get_confusion_matrix_for_model_and_data(pre_trained_model, x_test, y_test)

            two_confusion_matrices(
                from_scratch_confusion_matrix, 'Model trained from scratch',
                transfer_learned_confusion_matrix, f'Model pre-trained on {pre_train_ds_name}',
                dataset_name
            )


In [7]:
transfer_learning_for_all_datasets(USED_DATASETS)

Transfer Learning for all datasets:   0%|          | 0/3 [00:00<?, ?datasets/s]

Training model targeting dataset cbf


Pre-training on different datasets:   0%|          | 0/2 [00:00<?, ?datasets/s]

Pre-training model on dataset cbf


Training FCN on cbf dataset: 0epoch [00:00, ?epoch/s]

29/29 [==============================] - 0s 3ms/step - loss: 0.9076 - accuracy: 0.6933


Training FCN on distal_phalanax_tw dataset: 0epoch [00:00, ?epoch/s]

5/5 [==============================] - 0s 6ms/step - loss: 8.8902 - accuracy: 0.1295


Training FCN on cbf dataset: 0epoch [00:00, ?epoch/s]

ValueError: in user code:

    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\timwi\Documents\Uni\Ensem

In [22]:
x_train, y_train = USED_DATASETS['distal_phalanax_tw']['train_data']
x_test, y_test = USED_DATASETS['distal_phalanax_tw']['test_data']

pre_train_x_train, pre_train_y_train = USED_DATASETS['gun_point_male_female']['train_data']
pre_train_x_test, pre_train_y_test = USED_DATASETS['gun_point_male_female']['test_data']

dataset_name = 'distal_phalanax_tw'
pre_train_ds_name = 'gun_point_male_female'

In [23]:
# Training a model from scratch as a baseline:
scratch_model, scratch_loss, scratch_accuracy, scratch_history = train_model_from_scratch(
    x_train, y_train, x_test, y_test, dataset_name
)

Training FCN on distal_phalanax_tw dataset: 0epoch [00:00, ?epoch/s]

5/5 [==============================] - 0s 3ms/step - loss: 6.4912 - accuracy: 0.3022


In [24]:
# Pre-training a model:
pre_trained_model, _, _, _ = transfer_learning_model_one_dataset(
    pre_train_x_train, pre_train_y_train, pre_train_x_test, pre_train_y_test, pre_train_ds_name
)

Training FCN on gun_point_male_female dataset: 0epoch [00:00, ?epoch/s]

10/10 [==============================] - 0s 3ms/step - loss: 1.4875 - accuracy: 0.5918


In [49]:
# Fine-tuning:
input_size = x_train.shape[1]
output_size = len(np.unique(y_train))
headless_model_layers = pre_trained_model.layers[:-1]
fine_tuned_model = keras.Sequential([*headless_model_layers, keras.layers.Dense(output_size, activation='softmax')])
fine_tuned_model.summary()
# fine_tuned_model.build(input_shape=(None, input_size, 1))
train_single_model(
    fine_tuned_model, x_train, y_train, x_test, y_test,
    epochs=EPOCHS_PER_TRAINING, batch_size=25, model_name='FCN', dataset_name=dataset_name
)

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_24 (Conv1D)          (None, 150, 128)          1152      
                                                                 
 batch_normalization_24 (Bat  (None, 150, 128)         512       
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 150, 128)          0         
                                                                 
 conv1d_25 (Conv1D)          (None, 150, 256)          164096    
                                                                 
 batch_normalization_25 (Bat  (None, 150, 256)         1024      
 chNormalization)                                                
                                                                 
 activation_25 (Activation)  (None, 150, 256)        

Training FCN on distal_phalanax_tw dataset: 0epoch [00:00, ?epoch/s]

ValueError: in user code:

    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\timwi\Documents\Uni\EnsembleBasedTSC\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\timwi\Documents\Uni\Ensem

In [48]:
fine_tuned_model.output_shape

(None, 128)

In [41]:
def get_FCN2(input_size, output_size):
    """
    Create a Fully Convolutional Model based on
    Z. Wang, W. Yan, and T. Oates, “Time series classification from scratch with deep neural networks: A strong baseline”, 2017
    :param input_size: number of features of the input data
    :param output_size: number of classes
    :return: keras sequential model of the FCN
    """
    return keras.Sequential([
        keras.layers.Conv1D(filters=128, kernel_size=8, padding='same', input_shape=(input_size, 1), kernel_initializer=tf.keras.initializers.GlorotUniform()),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),

        keras.layers.Conv1D(filters=256, kernel_size=5, padding='same', kernel_initializer=tf.keras.initializers.GlorotUniform()),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),

        keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', kernel_initializer=tf.keras.initializers.GlorotUniform()),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),

        keras.layers.GlobalAveragePooling1D(),
        # keras.layers.Dense(output_size, activation='softmax', kernel_initializer=tf.keras.initializers.GlorotUniform()),
    ])

In [44]:
m = get_FCN2(11223, output_size)
m.build(input_shape=(None, input_size, 1))
m.output_shape

(None, 128)

In [ ]:
# Compare confusion matrices:
from_scratch_confusion_matrix = get_confusion_matrix_for_model_and_data(scratch_model, x_test, y_test)
transfer_learned_confusion_matrix = get_confusion_matrix_for_model_and_data(pre_trained_model, x_test, y_test)

two_confusion_matrices(
    from_scratch_confusion_matrix, 'Model trained from scratch',
    transfer_learned_confusion_matrix, f'Model pre-trained on {pre_train_ds_name}',
    dataset_name
)